In [2]:
%%time
#create DASK scheduler and worker remote containers
#this will take at least one minute as there are delays added on purpose to allow containers to spawn
#on successful run you should see scheduler URL printed

!python3 daskmaster.py

 Scheduler IP: 10.10.166.89
 Scheduler URL: tcp://10.10.166.89:8786


In [5]:

_0


NameError: name '_0' is not defined